<a href="https://colab.research.google.com/github/tengkuhaikal/asteroid/blob/main/asteroid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import pandas as pd
import heapq
import math

# --- LOAD ASTEROID DATA ---
asteroid_data = pd.read_csv('/content/drive/MyDrive/Asteroid_Field_Data.csv')

# --- CONFIG ---
grid_columns, grid_rows = 30, 30
safety_margin = 1.0

x_min, x_max = asteroid_data['X'].min(), asteroid_data['X'].max()
y_min, y_max = asteroid_data['Y'].min(), asteroid_data['Y'].max()
sector_width = (x_max - x_min) / grid_columns
sector_height = (y_max - y_min) / grid_rows

# --- BLOCKED GRID ---
blocked = np.zeros((grid_rows, grid_columns), dtype=bool)
for _, row in asteroid_data.iterrows():
    cx, cy, size = row['X'], row['Y'], row['Size']
    r = size / 2 + safety_margin
    for x in range(grid_columns):
        for y in range(grid_rows):
            x0 = x_min + x * sector_width
            y0 = y_min + y * sector_height
            x1 = x0 + sector_width
            y1 = y0 + sector_height
            closest_x = np.clip(cx, x0, x1)
            closest_y = np.clip(cy, y0, y1)
            dist = np.hypot(closest_x - cx, closest_y - cy)
            if dist < r:
                blocked[y, x] = True

# --- FIND NEAREST UNBLOCKED CELL ---
def find_nearest_unblocked(start_x, start_y, blocked):
    rows, cols = blocked.shape
    for r in range(max(rows, cols)):
        for dx in range(-r, r+1):
            for dy in range(-r, r+1):
                nx, ny = start_x + dx, start_y + dy
                if 0 <= nx < cols and 0 <= ny < rows:
                    if not blocked[ny, nx]:
                        return (nx, ny)
    return None

# --- FIND UNBLOCKED EDGE CELLS ---
def find_exit_on_edge(blocked, side='right'):
    rows, cols = blocked.shape
    candidates = []
    if side == 'right':
        for y in range(rows):
            if not blocked[y, cols - 1]:
                candidates.append((cols - 1, y))
    elif side == 'left':
        for y in range(rows):
            if not blocked[y, 0]:
                candidates.append((0, y))
    elif side == 'bottom':
        for x in range(cols):
            if not blocked[0, x]:
                candidates.append((x, 0))
    elif side == 'top':
        for x in range(cols):
            if not blocked[rows - 1, x]:
                candidates.append((x, rows - 1))
    return candidates

# --- EUCLIDEAN HEURISTIC ---
def heuristic(a, b):
    return math.hypot(a[0] - b[0], a[1] - b[1])

# --- A* PATHFINDER ---
def a_star(start, goal):
    rows, cols = blocked.shape
    gscore = np.full((rows, cols), np.inf)
    gscore[start[1], start[0]] = 0

    came_from = {}
    visited = np.zeros((rows, cols), dtype=bool)

    open_set = []
    heapq.heappush(open_set, (heuristic(start, goal), 0, start))

    while open_set:
        f, g, current = heapq.heappop(open_set)
        x, y = current

        if visited[y, x]:
            continue
        visited[y, x] = True

        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            path.reverse()
            return path

        for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < cols and 0 <= ny < rows:
                if blocked[ny, nx] or visited[ny, nx]:
                    continue
                tentative_g = g + math.hypot(dx, dy)
                if tentative_g < gscore[ny, nx]:
                    gscore[ny, nx] = tentative_g
                    came_from[(nx, ny)] = (x, y)
                    f_new = tentative_g + heuristic((nx, ny), goal)
                    heapq.heappush(open_set, (f_new, tentative_g, (nx, ny)))
    return None

# --- ENTRY & EXIT ---
start = find_nearest_unblocked(0, 0, blocked)

# New: combine right & bottom edge
right_candidates = find_exit_on_edge(blocked, side='right')
bottom_candidates = find_exit_on_edge(blocked, side='top')
exit_candidates = right_candidates + bottom_candidates

if exit_candidates:
    # Closest to bottom-right corner
    end = min(exit_candidates, key=lambda candidate: heuristic(candidate, (grid_columns - 1, grid_rows - 1)))
else:
    end = None

print(f"Using start: {start}, end: {end}")


safe_path = a_star(start, end) if start and end else None

# --- PLOT ---
fig, ax = plt.subplots(figsize=(12, 10))
scatter = ax.scatter(
    asteroid_data['X'], asteroid_data['Y'],
    c=asteroid_data['Size'],
    cmap='nipy_spectral_r',
    s=asteroid_data['Size'] * 5,
    alpha=1.0, edgecolor='black', linewidth=0.5
)

# Draw grid cells (optional)
for x in range(grid_columns):
    for y in range(grid_rows):
        sector_x = x_min + x * sector_width
        sector_y = y_min + y * sector_height
        color = 'black' if blocked[y, x] else 'white'
        rect = patches.Rectangle(
            (sector_x, sector_y), sector_width, sector_height,
            linewidth=0.2, edgecolor='gray', facecolor=color, alpha=0.5
        )
        ax.add_patch(rect)

# Draw path
if safe_path:
    path_points = [
        (x_min + x * sector_width + sector_width / 2,
         y_min + y * sector_height + sector_height / 2)
        for x, y in safe_path
    ]
    xs, ys = zip(*path_points)
    ax.plot(xs, ys, color='red', linewidth=2.5, marker='o', markersize=3, label='Safe Path')
else:
    print("No safe path found — fully blocked.")

plt.colorbar(scatter, label='Asteroid Size')
ax.set_title('Grid-based A* Path — Safe Entry & Forced Edge Exit')
ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.grid(True)
plt.tight_layout()
plt.legend()
plt.show()
